In [328]:
import os 
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO

In [329]:
SCORE_DIR = "data/scores"

In [330]:
box_scores = os.listdir(SCORE_DIR)

In [331]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [332]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()

    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup
        

In [333]:
def read_line_score(soup):
    line_score = pd.read_html(StringIO(str(soup)), attrs = {'id': 'line_score'})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    
    return line_score


In [334]:
def read_stats(soup, teams, stat):
    df = pd.read_html(StringIO(str(soup)) , attrs = {"id": f"box-{team}-game-{stat}"}, index_col = 0)[0]
    df = df.apply(pd.to_numeric, errors = "coerce")
    return df

In [335]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [336]:
base_cols = None
games =  []

for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score["team"])
    
    summaries = []
    for team in teams:
        basic = read_stats(soup, team , "basic")
        advanced = read_stats(soup, team, "advanced")
    
        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()
    
        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + "_max"
    
        summary = pd.concat([totals, maxes])
    
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep= "first"))
            base_cols = [b for b in base_cols if "bpm" not in b]
    
        summary = summary[base_cols]
    
        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T
    
    game = pd.concat([summary, line_score], axis = 1)
    game["home"] = [0,1]
    
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns +="_opp"
    
    full_game = pd.concat([game, game_opp], axis = 1)
    
    full_game["season"] = read_season_info(soup)
    
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"] , format="%Y%m%d")
    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)

    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

100 / 7588
200 / 7588
300 / 7588
400 / 7588
500 / 7588
600 / 7588
700 / 7588
800 / 7588
900 / 7588
1000 / 7588
1100 / 7588
1200 / 7588
1300 / 7588
1400 / 7588
1500 / 7588
1600 / 7588
1700 / 7588
1800 / 7588
1900 / 7588
2000 / 7588
2100 / 7588
2200 / 7588
2300 / 7588
2400 / 7588
2500 / 7588
2600 / 7588
2700 / 7588
2800 / 7588
2900 / 7588
3000 / 7588
3100 / 7588
3200 / 7588
3300 / 7588
3400 / 7588
3500 / 7588
3600 / 7588
3700 / 7588
3800 / 7588
3900 / 7588
4000 / 7588
4100 / 7588
4200 / 7588
4300 / 7588
4400 / 7588
4500 / 7588
4600 / 7588
4700 / 7588
4800 / 7588
4900 / 7588
5000 / 7588
5100 / 7588
5200 / 7588
5300 / 7588
5400 / 7588
5500 / 7588
5600 / 7588
5700 / 7588
5800 / 7588
5900 / 7588
6000 / 7588
6100 / 7588
6200 / 7588
6300 / 7588
6400 / 7588
6500 / 7588
6600 / 7588
6700 / 7588
6800 / 7588
6900 / 7588
7000 / 7588
7100 / 7588
7200 / 7588
7300 / 7588
7400 / 7588
7500 / 7588


In [337]:
games_df = pd.concat(games, ignore_index=True)

In [338]:
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,38.0,72.0,0.528,16.0,37.0,0.432,18.0,21.0,...,26.2,26.8,155.0,123.0,MIA,107,1,2023,2022-11-07,True
1,240.0,240.0,40.0,84.0,0.476,14.0,39.0,0.359,13.0,15.0,...,41.0,37.3,160.0,121.0,POR,110,0,2023,2022-11-07,False
2,240.0,240.0,41.0,78.0,0.526,8.0,24.0,0.333,15.0,19.0,...,28.6,41.1,250.0,125.0,DAL,90,1,2023,2022-12-14,True
3,240.0,240.0,29.0,74.0,0.392,13.0,38.0,0.342,19.0,26.0,...,12.6,33.0,183.0,110.0,CLE,105,0,2023,2022-12-14,False
4,240.0,240.0,37.0,87.0,0.425,7.0,33.0,0.212,32.0,35.0,...,20.0,32.3,226.0,117.0,TOR,126,1,2023,2022-12-07,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15171,240.0,240.0,43.0,92.0,0.467,12.0,34.0,0.353,12.0,19.0,...,51.5,33.0,200.0,112.0,MIN,117,0,2024,2023-11-10,False
15172,240.0,240.0,41.0,85.0,0.482,9.0,26.0,0.346,26.0,30.0,...,27.7,27.1,150.0,126.0,MIA,106,1,2020,2020-09-19,True
15173,240.0,240.0,33.0,85.0,0.388,12.0,44.0,0.273,28.0,34.0,...,51.5,36.2,141.0,114.0,BOS,117,0,2020,2020-09-19,False
15174,240.0,240.0,42.0,82.0,0.512,12.0,27.0,0.444,12.0,19.0,...,53.2,28.0,146.0,126.0,TOR,121,1,2024,2023-11-24,False


In [343]:
games_df.to_csv("nba_games.csv")